    !pip install ipykernel
    !pip install pandas
    !pip install plotly
    !pip install psycopg2
    !pip install --upgrade nbformat

In [108]:
from configparser import ConfigParser

import pandas as pd

import plotly.express as px

from psycopg2 import connect

In [109]:
def format_cursor(cursor):
    return [cursor.fetchall(),
             (desc[0] for desc in cursor.description)]

In [110]:
def data(data_in_query):
    return pd.DataFrame(data_in_query[0],
                        columns=data_in_query[1])

In [111]:
config = ConfigParser()
config.read('TEST.ini', encoding="utf-8")

config = {i: config['postgresql'][i]
           for i in config['postgresql']}

In [112]:
with connect(**config) as connect_1:
    with connect_1.cursor() as cursor_DAU:
        cursor_DAU.execute("""
        -- DAU (Daily Active Users) - Количество уникальных активных пользователей В ДЕНЬ
        -- WAU (Weekly Active Users) - Количество уникальных активных пользователей В НЕДЕЛЮ
        -- MAU (Monthly Active Users) - Количество уникальных активных пользователей В МЕСЯЦ

        -- DAU
        SELECT date_trunc('day', time) AS "date", COUNT(DISTINCT user_id) AS DAU FROM user_actions
        GROUP BY "date"
        """)
        df_DAU = data(format_cursor(cursor_DAU))

    with connect_1.cursor() as cursor_WAU:
        cursor_WAU.execute("""
        -- WAU (Поиск количества разделяя по понедельникам)
        SELECT date_trunc('week', time) "week",
        COUNT(DISTINCT user_id) WAU
        FROM user_actions
        GROUP BY "week"
        """)
        df_WAU = data(format_cursor(cursor_WAU))

    with connect_1.cursor() as cursor_MAU:
        cursor_MAU.execute("""
        -- MAU (Поиск количества разделяя по первым дням месяца)
        SELECT date_trunc('month', time) "month",
        COUNT(DISTINCT user_id) mau
        FROM user_actions
        GROUP BY "month"
        """)
        df_MAU = data(format_cursor(cursor_MAU))

    # Daily active couriers

    with connect_1.cursor() as cursor_DAC:
        cursor_DAC.execute("""
        SELECT COUNT(DISTINCT courier_id) as active_couriers,
        date_trunc('day', time) AS date
        FROM courier_actions
        GROUP BY 2
        """)
        df_DAC = data(format_cursor(cursor_DAC))

    # Daily active delivered orders

    with connect_1.cursor() as cursor_DADO:
        cursor_DADO.execute("""
        SELECT COUNT(order_id) as delivered_orders,
        date_trunc('day', time) AS date
        FROM courier_actions
        WHERE "action" LIKE 'del%'
        GROUP BY 2
        ORDER BY date
        """)
        df_DADO = data(format_cursor(cursor_DADO))

    with connect_1.cursor() as cursor_cancel_rate:
        cursor_cancel_rate.execute("""
        SELECT *,
        ROUND(cancel_orders / create_orders::DECIMAL, 3)::FLOAT as cancel_rate,
        ROUND(create_orders / active_users::DECIMAL, 3)::FLOAT as orders_per_users
        FROM (
            SELECT COUNT(order_id) FILTER(WHERE "action" LIKE 'can%') as cancel_orders,
            COUNT(order_id) FILTER(WHERE "action" LIKE 'crea%') as create_orders,
            COUNT(DISTINCT user_id) as active_users,
            date_trunc('day', time) AS date
            FROM user_actions
            GROUP BY date) t1
        """)
        df_cancel_rate = data(format_cursor(cursor_cancel_rate))

print(df_DAU.dtypes, df_WAU.dtypes, df_MAU.dtypes, df_DAC.dtypes, df_DADO.dtypes, df_cancel_rate.dtypes, sep="\n \n")

date    datetime64[ns]
dau              int64
dtype: object
 
week    datetime64[ns]
wau              int64
dtype: object
 
month    datetime64[ns]
mau               int64
dtype: object
 
active_couriers             int64
date               datetime64[ns]
dtype: object
 
delivered_orders             int64
date                datetime64[ns]
dtype: object
 
cancel_orders                int64
create_orders                int64
active_users                 int64
date                datetime64[ns]
cancel_rate                float64
orders_per_users           float64
dtype: object


In [ ]:
fig = px.line(df_DAU,
             x='date',
             y='dau',
             title='Daily Active Users',
             template='plotly_dark'
)

fig.update_traces(
    line_color=px.colors.qualitative.Alphabet[0],
    showlegend=True,
    name='Daily Active Users'
)

fig.update_layout(barmode='group',
                  showlegend=True,
                  legend_title='Legend'
)

fig.update_xaxes(title_text='')

Daily_Active_Users_Chart = fig

Daily_Active_Users_Chart.show()

In [ ]:
fig = px.line(df_WAU,
             x='week',
             y='wau',
             title='Weekly Active Users',
             template='plotly_dark'
)

fig.update_traces(
    line_color=px.colors.qualitative.Alphabet[1],
    showlegend=True,
    name='Weekly Active Users'
)

fig.update_layout(barmode='group',
                  showlegend=True,
                  legend_title='Legend'
)

fig.update_xaxes(title_text='')

Weekly_Active_Users_Chart = fig

fig.show()

In [ ]:
fig = px.line(df_MAU,
             x='month',
             y='mau',
             title='Monthly Active Users',
             template='plotly_dark'
)

fig.update_traces(
    line_color=px.colors.qualitative.Alphabet[2],
    showlegend=True,
    name='Monthly Active Users'
)

fig.update_layout(barmode='group',
                  showlegend=True,
                  legend_title='Legend'
)

fig.update_xaxes(title_text='')

Monthly_Active_Users_Chart = fig

Monthly_Active_Users_Chart.show()

In [ ]:
fig = px.line(df_DAC,
             x='date',
             y='active_couriers',
             title='Number of active couriers daily',
             template='plotly_dark'
)

fig.update_traces(
    line_color=px.colors.qualitative.Alphabet[3],
    showlegend=True,
    name='Number of active couriers daily'
)

fig.update_layout(barmode='group',
                  showlegend=True,
                  legend_title='Legend'
)

fig.update_xaxes(title_text='')

Number_of_active_couriers_daily = fig

Number_of_active_couriers_daily.show()

In [ ]:
fig = px.line(df_DADO,
             x='date',
             y='delivered_orders',
             title='Number of orders delivered daily',
             template='plotly_dark'
)

fig.update_traces(
    line_color=px.colors.qualitative.Alphabet[4],
    showlegend=True,
    name='Number of orders delivered daily'
)

fig.update_layout(barmode='group',
                  showlegend=True,
                  legend_title='Legend'
)

fig.update_xaxes(title_text='')

Number_of_orders_delivered_daily = fig

Number_of_orders_delivered_daily.show()

In [ ]:
fig = px.line(df_cancel_rate,
             x='date',
             y='cancel_rate',
             title='Cancel rate daily',
             template='plotly_dark'
)

fig.update_traces(
    line_color=px.colors.qualitative.Alphabet[5],
    showlegend=True,
    name='Cancel rate daily'
)

fig.update_layout(barmode='group',
                  showlegend=True,
                  legend_title='Legend'
)

fig.update_xaxes(title_text='')
fig.update_yaxes(rangemode='tozero')

Cancel_rate_daily_Chart = fig

Cancel_rate_daily_Chart.show()

In [ ]:
fig = px.line(df_cancel_rate,
             x='date',
             y='orders_per_users',
             title='Orders per users daily',
             template='plotly_dark'
)

fig.update_traces(
    line_color=px.colors.qualitative.Alphabet[6],
    showlegend=True,
    name='Orders per users daily'
)

fig.update_layout(barmode='group',
                  showlegend=True,
                  legend_title='Legend'
)

fig.update_xaxes(title_text='')
fig.update_yaxes(rangemode='tozero')

Orders_per_users_daily_Chart = fig

Orders_per_users_daily_Chart.show()

## Объединение всех графиков в одну картинку

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=4, cols=2)

fig.add_trace(Daily_Active_Users_Chart.data[0], row=1, col=1)
fig.add_trace(Weekly_Active_Users_Chart.data[0], row=1, col=2)
fig.add_trace(Monthly_Active_Users_Chart.data[0], row=2, col=1)
fig.add_trace(Number_of_active_couriers_daily.data[0], row=2, col=2)
fig.add_trace(Number_of_orders_delivered_daily.data[0], row=3, col=1)
fig.add_trace(Cancel_rate_daily_Chart.data[0], row=3, col=2)
fig.add_trace(Orders_per_users_daily_Chart.data[0], row=4, col=1)


fig.update_yaxes(title_text="dau", row=1, col=1)  # Replace with the appropriate titles and settings
fig.update_yaxes(title_text="wau", row=1, col=2)
fig.update_yaxes(title_text="mau", row=2, col=1)
fig.update_yaxes(title_text="active_couriers", row=2, col=2)
fig.update_yaxes(title_text="delivered_orders", row=3, col=1)


fig.update_yaxes(title_text='cancel_rate', rangemode='tozero', row=3, col=2)
fig.update_yaxes(title_text='orders_per_users', rangemode='tozero', row=4, col=1)

fig.update_layout(height=1000, width=1500, title_text="Что происходит? (Общая ситуация по заказам)", template='plotly_dark')
fig.show()
